<a href="https://colab.research.google.com/github/Byeongjuuuuu/nlp/blob/main/%EA%B0%90%EC%84%B1_%EB%B6%84%EB%A5%98task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import requirements

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import os
import pdb
import argparse
from dataclasses import dataclass, field
from typing import Optional
from collections import defaultdict

import torch
from torch.nn.utils.rnn import pad_sequence

import numpy as np
from tqdm import tqdm, trange

from transformers import (
    BertForSequenceClassification,
    BertTokenizer,
    AutoConfig,
    AdamW
)

# 1. Preprocess

In [ ]:
def make_id_file(task, tokenizer):
    def make_data_strings(file_name):
        data_strings = []
        with open(os.path.join(file_name), 'r', encoding='utf-8') as f:
            id_file_data = [tokenizer.encode(line.lower()) for line in f.readlines()]
        for item in id_file_data:
            data_strings.append(' '.join([str(k) for k in item]))
        return data_strings
    
    print('it will take some times...')
    train_pos = make_data_strings('sentiment.train.1')
    train_neg = make_data_strings('sentiment.train.0')
    dev_pos = make_data_strings('sentiment.dev.1')
    dev_neg = make_data_strings('sentiment.dev.0')

    print('make id file finished!')
    return train_pos, train_neg, dev_pos, dev_neg

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving sentiment.train.0 to sentiment.train.0
Saving sentiment.train.1 to sentiment.train.1


In [ ]:
!ls

dev_neg  dev_pos  sample_data  train_neg  train_pos


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 구글 드라이브에 파일을 업로드 해놓으셨다면 사용해주세요!

import pickle
with open("/content/drive/MyDrive/구름/batch32_sst2_model.bin", 'rb') as f:
    batch32_sst2_model.bin = pickle.load(f)
with open("/content/drive/MyDrive/구름/batch512_sst2_model.bin", 'rb') as f:
    batch512_sst2_model.bin = pickle.load(f)
with open("/content/drive/MyDrive/구름/test_no_label.csv", 'rb') as f:
    test_no_label.csv = pickle.load(f)
    '''
with open("/content/drive/MyDrive/1일차/dev_pos", 'rb') as f:
    dev_pos = pickle.load(f) 
    '''       

In [ ]:
train_pos[:10]

['101 6581 2833 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 2204 15174 2098 7570 22974 2063 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 2204 2326 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 1996 2047 2846 3504 6429 1012 102']

In [ ]:
train_pos, train_neg, dev_pos, dev_neg = make_id_file('yelp', tokenizer)

it will take some times...
make id file finished!


In [ ]:
# small 표본 데이터를 위함
import random
train_ran_pos = random.choices(train_pos, k=5000)
train_ran_neg = random.choices(train_neg, k=5000)
dev_ran_pos = random.choices(dev_pos, k=5000)
dev_ran_neg = random.choices(dev_neg, k=5000)

In [ ]:
train_pos[:10]

['101 6581 2833 1012 102',
 '101 21688 8013 2326 1012 102',
 '101 2027 2036 2031 3679 19247 1998 3256 6949 2029 2003 2428 2204 1012 102',
 '101 2009 1005 1055 1037 2204 15174 2098 7570 22974 2063 1012 102',
 '101 1996 3095 2003 5379 1012 102',
 '101 2204 3347 2833 1012 102',
 '101 2204 2326 1012 102',
 '101 11350 1997 2154 2003 25628 1998 7167 1997 19247 1012 102',
 '101 2307 2173 2005 6265 2030 3347 27962 1998 5404 1012 102',
 '101 1996 2047 2846 3504 6429 1012 102']

In [ ]:
class SentimentDataset(object):
    def __init__(self, tokenizer, pos, neg):
        self.tokenizer = tokenizer
        self.data = []
        self.label = []

        for pos_sent in pos:
            self.data += [self._cast_to_int(pos_sent.strip().split())]
            self.label += [[1]]
        for neg_sent in neg:
            self.data += [self._cast_to_int(neg_sent.strip().split())]
            self.label += [[0]]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample), np.array(self.label[index])

In [ ]:
train_dataset = SentimentDataset(tokenizer, train_pos, train_neg)
dev_dataset = SentimentDataset(tokenizer, dev_pos, dev_neg)

In [ ]:
# small 표본
train_dataset = SentimentDataset(tokenizer, train_ran_pos, train_ran_neg)
dev_dataset = SentimentDataset(tokenizer, dev_ran_pos, dev_ran_neg)

In [ ]:
# from transformers import DataCollatorWithPadding
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
# model = torch.load("/content/drive/MyDrive/1일차/default_model.bin", map_location='cpu')
# model.keys()
# from torch.utils.data import Subset
# import random
# indices = torch.randperm(len(train_dataset))[:10000]
# small_train_dataset = Subset(train_dataset, indices)
# small_dev_dataset = Subset(dev_dataset, indices)
# small_train_dataset_ = small_train_dataset.dataset
# small_dev_dataset_ = small_dev_dataset.dataset
# small_train_dataset_.label[1000]
# train_dataset[0]
# small_train_dataset[0]
# len(train_dataset)

In [ ]:
for i, item in enumerate(train_dataset):
    print(item)
    if i == 10:
        break

In [ ]:
# 설정한 batch_size를 기준으로 batch_size 개수의 문장 중 가장 긴 길이를 기준으로 padding

def collate_fn_style(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    # input len이 긴 것부터로 정렬하면 성능 조금 up
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    # sorted_indices = [i for i in range(len(input_ids))]

    attention_mask = torch.tensor(
        [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
         sorted_indices])
    
    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [ ]:
len(train_dataset)

443259

sst2 데이터 준비

In [ ]:
!pip install datasets
from sklearn.model_selection import train_test_split
from datasets import load_dataset
from datasets import load_metric
my_dataset = load_dataset("glue", "sst2")
my_metric = load_metric("glue", "sst2")
my_metric_2 = load_metric("accuracy")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 20.2 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-4-0311e135e458>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  my_metric = load_metric("glue", "sst2")


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding=False, truncation=True)

tokenized_datasets = my_dataset.map(tokenize_function, batched=True)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
def data_to_numpy(examples):
    return np.array(examples['input_ids']), np.array(examples['label'])
train_dataset = list()
for i in range(len(tokenized_datasets['train'])):
    train_dataset.append(data_to_numpy(tokenized_datasets['train'][i]))
test1 = list()
for i in range(len(tokenized_datasets['validation'])):
    test1.append(data_to_numpy(tokenized_datasets['validation'][i]))
train, test2, _ , _ = train_test_split(train_dataset, train_dataset , test_size = 0.1, random_state = 42)
train, val, _ , _ = train_test_split(train, train , test_size = 0.05, random_state = 42)
test = test1 + test2

In [ ]:
print(len(train))
print(len(val))
print(len(test))
print(train[0])
print(val[0])
print(test[0])

57583
3031
7607
(array([  101,  8478,  2784, 18495,  1997, 12039,   102]), array(1))
(array([  101,  2070,  1997,  1996,  4569, 15580,  2102, 13198,  1997,
        2151,  3185,   102]), array(1))
(array([  101,  2009,  1005,  1055,  1037, 11951,  1998,  2411, 12473,
        4990,  1012,   102]), array(1))


In [ ]:
dev_dataset = list()
test_dataset = list()

train_dataset = train
dev_dataset = val
test_dataset = test

sst2 데이터 코드 마무리

In [ ]:
# 배치 설정!

train_batch_size=256
eval_batch_size=128

train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=train_batch_size,
                                           shuffle=True, collate_fn=collate_fn_style,
                                           pin_memory=True, num_workers=2)
dev_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=eval_batch_size,
                                         shuffle=False, collate_fn=collate_fn_style,
                                         num_workers=2)

In [ ]:
# for sst2 / small_train_dataset, small_val_dataset

# train_batch_size=256
# eval_batch_size=512

# train_loader = torch.utils.data.DataLoader(small_train_dataset,
#                                            batch_size=train_batch_size,
#                                            shuffle=True, collate_fn=collate_fn_style,
#                                            pin_memory=True, num_workers=2)
# dev_loader = torch.utils.data.DataLoader(small_val_dataset, batch_size=eval_batch_size,
#                                          shuffle=False, collate_fn=collate_fn_style,
#                                          num_workers=2)

In [ ]:
# wandb login
!pip install wandb==0.9.7
import wandb
wandb.login()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 KB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 KB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 KB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.3 MB/s eta 0:00:00
  Created wheel for gql: filename=gql-0.2.0-py3-none-any.whl size=7639 sha256=90cf9522b00c394bdea815da773c36868c20f58a0961e4c25821ffb0f0b3a372
  Stored in directory: /root/.ca

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# for single learning
# random seed
random_seed=42
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import torch.optim as optim
model.train()
learning_rate = 5e-5

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100, eta_min=0)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# # for multiple learning
# # random seed
# random_seed=42
# np.random.seed(random_seed)
# torch.manual_seed(random_seed)

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model1 = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# model2 = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# model3 = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# model4 = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# model5 = BertForSequenceClassification.from_pretrained('bert-base-uncased')
# model=[model1, model2, model3, model4, model5]

# for i in range(5):
#     model[i].to(device)
# for i in range(5):
#     model[i].train()
# # learning_rate = 5e-5
# learning_rate = [5e-5, 4e-5, 3e-5, 2e-5, 1e-5]
# optimizer1 = AdamW(model1.parameters(), lr=learning_rate[0])
# optimizer2 = AdamW(model2.parameters(), lr=learning_rate[1])
# optimizer3 = AdamW(model3.parameters(), lr=learning_rate[2])
# optimizer4 = AdamW(model4.parameters(), lr=learning_rate[3])
# optimizer5 = AdamW(model5.parameters(), lr=learning_rate[4])
# optimizer = [optimizer1,optimizer2,optimizer3,optimizer4,optimizer5]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
def compute_acc(predictions, target_labels):
    return (np.array(predictions) == np.array(target_labels)).mean()

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
wandb.init(project='SST2 Batch size Project', name='batch 256', entity='252525')
train_epoch = 3
lowest_valid_loss = 9999.
for epoch in range(train_epoch):
    with tqdm(train_loader, unit="batch") as tepoch:
        for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
            tepoch.set_description(f"Epoch {epoch}")            
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            token_type_ids = token_type_ids.to(device)
            position_ids = position_ids.to(device)
            labels = labels.to(device, dtype=torch.long)            

            optimizer.zero_grad()

            output = model(input_ids=input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids,
                          position_ids=position_ids,
                          labels=labels)

            loss = output.loss
            loss.backward()
            
            optimizer.step()
            scheduler.step()            

            tepoch.set_postfix(loss=loss.item())
            if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
                # Evaluate the model five times per epoch
                with torch.no_grad():
                    model.eval()
                    valid_losses = []
                    predictions = []
                    target_labels = []
                    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
                                                                                                desc='Eval',
                                                                                                position=1,
                                                                                                leave=None):
                        input_ids = input_ids.to(device)
                        attention_mask = attention_mask.to(device)
                        token_type_ids = token_type_ids.to(device)
                        position_ids = position_ids.to(device)
                        labels = labels.to(device, dtype=torch.long)

                        output = model(input_ids=input_ids,
                                      attention_mask=attention_mask,
                                      token_type_ids=token_type_ids,
                                      position_ids=position_ids,
                                      labels=labels)

                        logits = output.logits
                        loss = output.loss
                        valid_losses.append(loss.item())

                        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
                        batch_labels = [int(example) for example in labels]

                        predictions += batch_predictions
                        target_labels += batch_labels

                acc = compute_acc(predictions, target_labels)                
                valid_loss = sum(valid_losses) / len(valid_losses)
                wandb.log({"Acc": acc}) # acc
                wandb.log({"Valid_loss": valid_loss}) # valid loss       
                if lowest_valid_loss > valid_loss:
                    print('Acc for model which have lower valid loss: ', acc)
                    lowest_valid_loss = valid_loss
                    torch.save(model.state_dict(), "./batch256_sst2_model.bin")                    
            wandb.log({"Loss": loss}) # test loss

wandb: Wandb version 0.13.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
Eval: 100%|██████████| 24/24 [00:07<00:00,  3.40it/s]
                                                     

Acc for model which have lower valid loss:  0.9135598812273177


Eval: 100%|██████████| 24/24 [00:07<00:00,  3.42it/s]
                                                     

Acc for model which have lower valid loss:  0.9254371494556252


Eval: 100%|██████████| 24/24 [00:07<00:00,  3.43it/s]
                                                     

...

Eval: 100%|██████████| 24/24 [00:07<00:00,  3.42it/s]
                                                     

Acc for model which have lower valid loss:  0.9468822170900693


Eval: 100%|██████████| 24/24 [00:07<00:00,  3.44it/s]
                                                     

Acc for model which have lower valid loss:  0.9498515341471462


Epoch 2: 100%|██████████| 225/225 [07:22<00:00,  1.97s/batch, loss=0.05]


## 일반 모델 성능 평가

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
pre = torch.load("/content/drive/MyDrive/구름/batch32_sst2_model.bin")
model.load_state_dict(pre)
model.to(device)

In [ ]:
import pandas as pd
test_df = pd.read_csv('/content/drive/MyDrive/구름/test_no_label.csv')

In [ ]:
test_dataset = test_df['Id']

In [ ]:
def make_id_file_test(tokenizer, test_dataset):
    data_strings = []
    id_file_data = [tokenizer.encode(sent.lower()) for sent in test_dataset]
    for item in id_file_data:
        data_strings.append(' '.join([str(k) for k in item]))
    return data_strings

In [ ]:
test = make_id_file_test(tokenizer, test_dataset)

In [ ]:
test[:2]

['101 2009 1005 1055 1037 2878 2047 3325 1998 2047 26389 2169 2051 2017 2175 1012 102',
 '101 2061 15640 2013 2019 2214 5440 1012 102']

In [ ]:
class SentimentTestDataset(object):
    def __init__(self, tokenizer, test):
        self.tokenizer = tokenizer
        self.data = []

        for sent in test:
            self.data += [self._cast_to_int(sent.strip().split())]

    def _cast_to_int(self, sample):
        return [int(word_id) for word_id in sample]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        sample = self.data[index]
        return np.array(sample)

In [ ]:
test_dataset = SentimentTestDataset(tokenizer, test)

In [ ]:
def collate_fn_style_test(samples):
    input_ids = samples
    max_len = max(len(input_id) for input_id in input_ids)
    # sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    sorted_indices = [i for i in range(len(input_ids))]

    attention_mask = torch.tensor(
          [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
          sorted_indices])

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    # label= 우리가 inference 해야 하기 때문에 없다.

    return input_ids, attention_mask, token_type_ids, position_ids

In [ ]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions += batch_predictions        

In [ ]:
test_df['Category'] = predictions

In [ ]:
test_df.to_csv('Batch 32_.csv', index=False)

##SST2 성능 평가

In [ ]:
!pip install datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from datasets import load_dataset
from datasets import load_metric
my_dataset = load_dataset("glue", "sst2")
my_metric = load_metric("glue", "sst2")
my_metric_2 = load_metric("accuracy")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# 최고 성능 모델을 사용하기 위해 모델 불러오기

model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
pre = torch.load("/content/drive/MyDrive/구름/batch512_sst2_model.bin")
model.load_state_dict(pre)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
import torch

In [ ]:
# for sst2

def collate_fn_style_test(samples):
    input_ids, labels = zip(*samples)
    max_len = max(len(input_id) for input_id in input_ids)
    sorted_indices = np.argsort([len(input_id) for input_id in input_ids])[::-1]
    # sorted_indices = [i for i in range(len(input_ids))]

    attention_mask = torch.tensor(
          [[1] * len(input_ids[index]) + [0] * (max_len - len(input_ids[index])) for index in
          sorted_indices])

    input_ids = pad_sequence([torch.tensor(input_ids[index]) for index in sorted_indices],
                             batch_first=True)
    
    token_type_ids = torch.tensor([[0] * len(input_ids[index]) for index in sorted_indices])
    position_ids = torch.tensor([list(range(len(input_ids[index]))) for index in sorted_indices])
    labels = torch.tensor(np.stack(labels, axis=0)[sorted_indices])

    return input_ids, attention_mask, token_type_ids, position_ids, labels

In [ ]:
test_batch_size = 32
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=test_batch_size,
                                          shuffle=False, collate_fn=collate_fn_style_test,
                                          num_workers=2)

In [ ]:
# for sst2
f1 = []

with torch.no_grad():
    model.eval()
    predictions = []
    for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(test_loader,
                                                                        desc='Test',
                                                                        position=1,
                                                                        leave=None):

        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        token_type_ids = token_type_ids.to(device)
        position_ids = position_ids.to(device)

        output = model(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids,
                       position_ids=position_ids)

        logits = output.logits
        batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
        predictions = batch_predictions
        my_metric.add_batch(predictions=predictions, references=labels)
        f1.append(f1_score(y_true=labels, y_pred=predictions).item())
        # my_metric_2.add_batch(predictions=predictions, references=labels)


Test:  99%|█████████▉| 236/238 [00:09<00:00, 40.35it/s]
                                                       

In [ ]:
# for sst2
#batch_size = 512
print(my_metric.compute())
# print(my_metric_2.compute())

{'accuracy': 0.9462337320888655}


In [ ]:
length = len(f1)
f1 = sum(f1) / length

In [ ]:
print(f'F-1 score: {f1}')

F-1 score: 0.9494225593771968


##sst2 test compute

In [ ]:
!pip install datasets
from datasets import load_dataset
from datasets import load_metric
my_dataset = load_dataset("glue", "sst2")
my_metric = load_metric("glue", "sst2")

In [ ]:
my_dataset['train'][0]

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding=False, truncation=True)

tokenized_datasets = my_dataset.map(tokenize_function, batched=True)

In [ ]:
def data_to_numpy(examples):
    return np.array(examples['input_ids']), np.array(examples['label'])
tokenized_datasets['train'][0]

tokenized_datasets['train']['idx'][-1] # 67348
tokenized_datasets['validation']['idx'][-1] # 871
tokenized_datasets['test']['idx'][-1] # 1820

In [ ]:
small_train = tokenized_datasets['train'].shuffle(seed=42).select(range(10000))
small_train[0]

In [ ]:
small_train_dataset = list()
for i in range(len(small_train)):
    small_train_dataset.append(data_to_numpy(small_train[i]))
small_val_dataset = list()
for i in range(len(tokenized_datasets['validation'])):
    small_val_dataset.append(data_to_numpy(tokenized_datasets['validation'][i]))

In [ ]:
tokenized_datasets['test'][0]

{'sentence': 'uneasy mishmash of styles and genres .',
 'label': -1,
 'idx': 0,
 'input_ids': [101,
  15491,
  28616,
  22444,
  4095,
  1997,
  6782,
  1998,
  11541,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
# reps = 5
# for rep in range(3, reps):
#     run = wandb.init(project='Project2', name=f'lr{reps - rep}', reinit=True)
#     train_epoch = 3
#     lowest_valid_loss = 9999.
#     for epoch in range(train_epoch):
#         with tqdm(train_loader, unit="batch") as tepoch:
#             for iteration, (input_ids, attention_mask, token_type_ids, position_ids, labels) in enumerate(tepoch):
#                 tepoch.set_description(f"Epoch {epoch}")            
#                 input_ids = input_ids.to(device)
#                 attention_mask = attention_mask.to(device)
#                 token_type_ids = token_type_ids.to(device)
#                 position_ids = position_ids.to(device)
#                 labels = labels.to(device, dtype=torch.long)

#                 optimizer[rep].zero_grad()

#                 output = model[rep](input_ids=input_ids,
#                               attention_mask=attention_mask,
#                               token_type_ids=token_type_ids,
#                               position_ids=position_ids,
#                               labels=labels)

#                 loss = output.loss
#                 loss.backward()

#                 optimizer[rep].step()

#                 tepoch.set_postfix(loss=loss.item())            
#                 if iteration != 0 and iteration % int(len(train_loader) / 5) == 0:
#                     # Evaluate the model five times per epoch
#                     with torch.no_grad():
#                         model[rep].eval()
#                         valid_losses = []
#                         predictions = []
#                         target_labels = []
#                         for input_ids, attention_mask, token_type_ids, position_ids, labels in tqdm(dev_loader,
#                                                                                                     desc='Eval',
#                                                                                                     position=1,
#                                                                                                     leave=None):
#                             input_ids = input_ids.to(device)
#                             attention_mask = attention_mask.to(device)
#                             token_type_ids = token_type_ids.to(device)
#                             position_ids = position_ids.to(device)
#                             labels = labels.to(device, dtype=torch.long)

#                             output = model[rep](input_ids=input_ids,
#                                           attention_mask=attention_mask,
#                                           token_type_ids=token_type_ids,
#                                           position_ids=position_ids,
#                                           labels=labels)

#                             logits = output.logits
#                             loss = output.loss
#                             valid_losses.append(loss.item())

#                             batch_predictions = [0 if example[0] > example[1] else 1 for example in logits]
#                             batch_labels = [int(example) for example in labels]

#                             predictions += batch_predictions
#                             target_labels += batch_labels

#                     acc = compute_acc(predictions, target_labels)                
#                     valid_loss = sum(valid_losses) / len(valid_losses)
#                     wandb.log({"Acc": acc}) # acc
#                     wandb.log({"Valid_loss": valid_loss}) # valid loss       
#                     if lowest_valid_loss > valid_loss:
#                         print('Acc for model which have lower valid loss: ', acc)                    
#                         torch.save(model[rep].state_dict(), f"./lr{reps - rep}, batch64_model.bin")
#                 wandb.log({"Loss": loss}) # test loss